In [ ]:
import requests
import pandas as pd
import json

In [ ]:
api_key_info = json.load( open('C:/Users/Dharik/lc_api.json') )['normal']
investor_id = api_key_info['investor_id']
auth = api_key_info['auth']

# Get all my current notes

In [ ]:
r = requests.get('https://api.lendingclub.com/api/investor/v1/accounts/' + investor_id + '/detailednotes', headers={'Authorization': auth, 'Accept':'application/json'}) 
my_notes = pd.DataFrame( r.json().get('myNotes') )
my_notes.set_index( 'loanId', inplace=True )

In [ ]:
print( 'Got', len(my_notes), 'notes')

# Sell any bad notes

In [ ]:
bad_notes = my_notes.query('(currentPaymentStatus == "Overdue" | currentPaymentStatus == "Not Received") & canBeTraded == True')

In [ ]:
print( 'There are', len(bad_notes), 'notes that are late')

pd.set_option('display.max_columns', None)
bad_notes

### Set some discounted price

In [ ]:
notes_to_sell = []
for loanId,rowData in bad_notes.iterrows():
    remainingValue = rowData['principalPending'] + rowData['accruedInterest']
    
    if rowData['loanStatus'].find('Grace') > 0:
        askPrice = remainingValue *.8
    elif rowData['loanStatus'].find('120') > 0:
        askPrice = remainingValue * .3
    else:
        askPrice = remainingValue * .6
        
    print( 'Status:', rowData['loanStatus'], 'Remaining:', remainingValue, 'Ask Price:', askPrice)
    
    notes_to_sell.append({'loanId':int(loanId), "orderId": rowData['orderId'], "noteId": rowData['noteId'], "askingPrice": askPrice})

notes_to_sell

In [ ]:
import datetime 

expire_date = datetime.datetime.now() + datetime.timedelta(days=4)

sell_request = {"aid": investor_id, "expireDate": expire_date.strftime('%m/%d/%Y'), "notes":notes_to_sell}

sell_request

### Submit sell request

In [ ]:
r = requests.post('https://api.lendingclub.com/api/investor/v1/accounts/' + investor_id + '/trades/sell', headers={'Authorization': auth, 'Accept':'application/json'}, json=sell_request) 

In [ ]:
from pprint import pprint
pprint(r.json())